In [73]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq


In [74]:
country_list = pd.read_csv('./data/CensusCountryMapping_FRBAtl_short.csv', dtype={'CTY_CODE': str})

country_list.loc[len(country_list)] = ["0003", "EUE", "EUROPEAN UNION"]

country_list.loc[len(country_list)] = ["5082", "GZA", "Gaza"]

country_list.loc[len(country_list)] = ["5083", "WBK", "West Bank"]

country_list = country_list.dropna()

country_list["tariff"] = 10.0

country_list.loc[country_list["CTY_CODE"] == "5700", "tariff"] = 30.0

country_list.loc[country_list["CTY_CODE"] == "5820", "tariff"] = 30.0

country_list.loc[country_list["CTY_CODE"] == "1220", "tariff"] = 8.75

country_list.loc[country_list["CTY_CODE"] == "2010", "tariff"] = 4.5

country_list.loc[country_list["CTY_CODE"] == "0003", "tariff"] = 10.0
#IF EU you get 50 and you'll like it


In [75]:
country_list[country_list["country"] == 'Palestine']

,CTY_CODE,ISO3,country,tariff


In [76]:
expemption_list = pd.read_csv('./tariff-lists/annex-II-exemptions.csv', dtype={'HTSUS': str})
# these are the annex II exemptions from the reciprical tariff list

expemption_list_phones = pd.read_csv('./tariff-lists/41225_exemptions.csv', dtype={'HTSUS': str})
# these are the phone exemptions

steel_list = pd.read_csv('./tariff-lists/steel-tariffs.csv', dtype={'HTSUS': str})
# these are the steel products subject to the new 232 tariffs. This is at 8 digit level.

alu_list = pd.read_csv('./tariff-lists/alu-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

alu_8_list = alu_list[alu_list['HTSUS'].str.len() == 8]

auto_list = pd.read_csv('./tariff-lists/auto-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

auto_6_list = auto_list[auto_list['HTSUS'].str.len() == 6]

auto_8_list = auto_list[auto_list['HTSUS'].str.len() == 8]


# country_list = pd.read_csv('./tariff-lists/reciprocal-tariffs-annex-I.csv',dtype={'CTY_CODE': str})

In [77]:
def weighted_average(df, value_col, weight_col):
    """
    Compute the weighted average of `value_col` using `weight_col` as weights.
    """
    return (df[value_col] * df[weight_col]).sum() / df[weight_col].sum()

In [78]:
def get_all_imports(cnty_code, tariff):

    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    hs = 'HS10'

    end_use = "hs?get=CTY_NAME,CON_VAL_YR,CAL_DUT_YR,I_COMMODITY,I_COMMODITY_SDESC"

    surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

    surl  = surl + my_key + "&time=" + "2024-12" + "&COMM_LVL=" + hs 

    url = surl + "&CTY_CODE=" + cnty_code

    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    #######################################################################

    df.time = pd.to_datetime(df.time, format="%Y-%m")

    df["imports"] = df["CON_VAL_YR"].astype(float)

    df["duty"] = df["CAL_DUT_YR"].astype(float)

    df["share"] = df["imports"] / df["imports"].sum()
            
    df[hs] = df["I_COMMODITY"].astype(str)

    df["HS8"] = df["HS10"].str[0:8]

    df["HS6"] = df["HS10"].str[0:6]

    df["HS2"] = df["HS10"].str[0:2]

    #######################################################################
    # this then filters so products are only those that are subject to the new tariffs
    # the reciprocal exemptions, steel, aluminum and auto tariffs.

    df["tariff"] = tariff

    exemption = df["HS10"].isin(expemption_list["HTSUS"].tolist()) 

    df.loc[exemption, "tariff"]  = 0.0

    exempt_imports = df.loc[exemption, "imports"].sum()

    ####################################################
    # this is the phone exemption list. This is a 10 digit list.

    exemption_phones = df["HS10"].isin(expemption_list_phones["HTSUS"].tolist()) 

    if cnty_code == "5700": # if china then still Fentanyl 20 percent on the phones

        df.loc[exemption_phones, "tariff"] = 20.0

    else:

        df.loc[exemption_phones, "tariff"] = 0.0

    ####################################################

    steel = df["HS8"].isin(steel_list["HTSUS"].tolist())

    if cnty_code == "4120": # if the UK, then exempted from the increase

        df.loc[steel, "tariff"]  = 25.0 

    else:

        df.loc[steel, "tariff"]  = 50.0 


    steel_imports = df.loc[steel, "imports"].sum()

    ####################################################

    alu = df["HS10"].isin(alu_list["HTSUS"].tolist()) | df["HS8"].isin(alu_8_list["HTSUS"].tolist())

    if cnty_code == "4120": # if the UK, then exempted from the increase
       
        df.loc[alu, "tariff"]  = 25.0 

    else:

        df.loc[alu, "tariff"]  = 50.0
        
    alu_imports = df.loc[alu, "imports"].sum()

    ####################################################


    auto = df["HS8"].isin(auto_8_list["HTSUS"].tolist()) | df["HS10"].isin(auto_list["HTSUS"].tolist()) | df["HS6"].isin(auto_6_list["HTSUS"].tolist())

    if cnty_code != "1220" and cnty_code != "2010": # if canada or mexico, then assumption is no tariff

        df.loc[auto, "tariff"]  = 25.0

    auto_imports = df.loc[auto, "imports"].sum()

    ####################################################


    return df

In [79]:
f'./data-hs2/hs2-tariff-{row["CTY_CODE"]}.csv'


'./data-hs2/hs2-tariff-0003.csv'

In [80]:
row['country']

'EUROPEAN UNION'

In [81]:
for index, row in country_list.iterrows():

    print(f"Processing {row['country']}")

    out_file = f'./data-hs2/hs2-tariff-{row["CTY_CODE"]}.csv'

    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    df = get_all_imports(row["CTY_CODE"], row["tariff"])

    dfhs2 = df.groupby("HS2").agg({"imports": "sum", "duty": "sum", "CTY_NAME": "first", 
                              "CTY_CODE": "first"}).reset_index()

    foo = df.groupby("HS2").apply(lambda x: weighted_average(x, "tariff", "imports"))

    dfhs2 = dfhs2.merge(foo.rename("2025_tariff_increase"), left_on="HS2", right_index=True)

    dfhs2.rename(columns={"imports": "2024_imports", "duty": "2024_duty"}, inplace=True)

    dfhs2["2024_tariff"] = 100* ( dfhs2["2024_duty"] / dfhs2["2024_imports"] )

    dfhs2.to_csv(out_file, index=False)

Processing Andorra
Already have downloaded file
Processing United Arab Emirates
Already have downloaded file
Processing Afghanistan
Already have downloaded file
Processing Antigua and Barbuda
Already have downloaded file
Processing Anguilla
Already have downloaded file
Processing Albania
Already have downloaded file
Processing Armenia
Already have downloaded file
Processing Angola
Already have downloaded file
Processing Argentina
Already have downloaded file
Processing Austria
Already have downloaded file
Processing Australia
Already have downloaded file
Processing Aruba
Already have downloaded file
Processing Azerbaijan
Already have downloaded file
Processing Bosnia and Herzegovina
Already have downloaded file
Processing Barbados
Already have downloaded file
Processing Bangladesh
Already have downloaded file
Processing Belgium
Already have downloaded file
Processing Burkina Faso
Already have downloaded file
Processing Bulgaria
Already have downloaded file
Processing Bahrain
Already ha

In [82]:
checkdf = pd.read_csv(f'./data-hs2/hs2-tariff-0003.csv')

In [83]:
(checkdf["2025_tariff_increase"] * checkdf["2024_imports"]).sum() / checkdf["2024_imports"].sum()

12.17140233454136

In [84]:
checkdf["2024_duty"].sum() / checkdf["2024_imports"].sum()

0.012290777517549551